<a href="https://colab.research.google.com/github/UITTrinhQuangTruong/CS114.K21/blob/master/%C4%90%E1%BB%93_%C3%A1n_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Đổi tên hàng loạt

In [ ]:
#405 Cobaoluc
#412 Khongbaoluc
!pip install hickle

#Tiền xử lý Dữ liệu

## Import các thư viện cần thiết

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from skimage.feature import hog
from sklearn.cluster import KMeans
from os import listdir
from os.path import isfile, join

##Trích xuất Optical flow bằng thuật toán TVL1 có trong OpenCV
 

### Định nghĩa về Optical flow
Optical Flow là khái niệm chỉ sự chuyển động tương đối của các điểm trên bề mặt một đối tượng, vật thể nào đó gây ra, dưới góc quan sát của một điểm mốc (mắt, camera…).

Dễ hiểu là biểu diễn điểm khác biệt giữa 2 khung hình do vật thể di chuyển gây ra.

![Hình ảnh mô tả Optical flow của các xe](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/opticalflow_lk.jpg)

###Các bước thực hiện
B1: Định nghĩa đối tượng bằng hàm DualTVL1OpticalFlow_create()

B2: Tính flowx, flowy của 2 frame gần nhau bằng hàm calc. Với flowx là Optical Flow theo hướng x và tương tự với y

B3: Tính mag = sqrt(flowx + flowy)

B4: Normalize mag về ảnh hsv

B5: Chuyển ảnh về định dạng Grayscale

In [ ]:
optical_flow = cv2.optflow_DualTVL1OpticalFlow.create()
def OpticalFlow(prvs, next, hsv):
    
    flow = optical_flow.calc(prvs, next, None)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    nom = np.zeros((160,120))
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)

    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    return gray

##Trích xuất vùng chuyển động
Từ ảnh Optimal Flow vừa tìm được ta trích xuất ra vùng có sự chuyển động


###Phát hiện cạnh với thuật toán Canny
![Cận trên và dứoi](https://docs.opencv.org/trunk/hysteresis.jpg)

Thuật toán Canny tham khảo thêm [tại đây](https://docs.opencv.org/trunk/da/d22/tutorial_py_canny.html)

Hàm tìm maxVal và minVal tham khảo thêm [tại đây](https://www.pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv/)

In [ ]:
def CvCanny(matrix):
  v = np.median(matrix)
  sigma = 0.33
  lower = int(max(0, (1.0 - sigma) * v))
  upper = int(min(255, (1.0 + sigma) * v))
  canny = cv2.Canny(matrix, lower, upper)
  return canny

###Trích xuất vùng chuyển động

In [ ]:
def MRM(prvs, next, hsv):
    #Trích xuất dòng quang từ 2 frame gần nhau
    gray = OpticalFlow(prvs, next, hsv)
    '''if s == 3:
    flowx = cv2.normalize(flow[...,0],None,0,255,cv2.NORM_MINMAX)
    cv2_imshow(flowx)
    flowy = cv2.normalize(flow[...,1],None,0,255,cv2.NORM_MINMAX)
    cv2_imshow(flowy)'''
    '''Tăng độ nét cho ảnh dòng quang
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpen = cv2.filter2D(hsv, -1, kernel)
    canny1 = CvCanny(sharpen)'''
    #Phát hiện cạnh
    canny = CvCanny(gray)
    #gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    #cv2_imshow(gray)    
    '''gray1 = cv2.filter2D(gray, -1, kernel)[...,::-1]
    gray = cv2.ximgproc.guidedFilter(gray, gray1, radius=2, eps=0.004, dDepth=-1)[...,::-1]
    canny3 = CvCanny(gray)
    cv2_imshow(gray)'''
    #Đóng các nét hở và lấp đầy
    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
    '''h, w = closing.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    im_floodfill = closing.copy()
    cv2.floodFill(im_floodfill, mask, (0,0), 255);
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    floodfill = closing | im_floodfill'''

    #Lấp đầy những lỗ hổng trong vùng chuyển động
    contour,hier = cv2.findContours(closing,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contour:
        cv2.drawContours(closing,[cnt],0,255,-1)
    
    #Xóa bỏ những nét thừa
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    
    return opening

#Trích xuất đặc trưng


##Vùng quan tâm

Xem vùng chuyển động như là một vùng quan tâm (ROI), áp dụng ROI vào ảnh xám

In [ ]:
def apply_roi(img, roi):
  #thresh, roi = cv2.threshold(roi, thresh=128, maxval=1, type=cv2.THRESH_BINARY)
  new_img = img * roi
  return new_img

##Trích xuất đặc trưng

In [ ]:
def extract_features(path, label, descriptor_list, labels):
    #Lấy danh sách các clip trong path 
    videoPaths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    for videoPath in videoPaths:

        #Đọc video
        cap = cv2.VideoCapture(videoPath)
        videoPaths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]

        ret, frame1 = cap.read()
        if not ret:
          break;
        #Thay đổi kích thước frame
        frame1 = cv2.resize(frame1, (160, 120))
        prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
        hsv = np.zeros_like(frame1)
        hsv[...,1] = 255
        s = 1
        while(cap.isOpened()):
            ret, frame2 = cap.read()

            if not ret:
                break;

            if s % 6 == 0:
                frame2 = cv2.resize(frame2, (160, 120))
                next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
                mrm = MRM(prvs, next, hsv)

                after_roi = apply_roi(next, mrm)
                descriptor = hog(after_roi, orientations=12, pixels_per_cell=(4, 4), cells_per_block=(2,2), block_norm='L2-Hys', visualize=False)
                descriptor_list.append(descriptor)
                labels.append(label)
                prvs = next
            s += 1

        print(videoPath + "---done--")


In [ ]:
labels = []
descriptor_list = []

In [ ]:
path = "/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc"
extract_features(path, 0, descriptor_list, labels)
path = "/content/drive/My Drive/Machine Lol/Data/Của Trường/khongbaoluc"
extract_features(path, 1, descriptor_list, labels)

/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/136.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/137.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/138.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/139.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/140.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/141.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/142.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/143.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/144.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/145.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/146.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/147.mp4---done--
/content/drive/M

In [ ]:
print(len(descriptor_list))
print(len(labels))

3584
3584


##Chuẩn hóa đặc trưng

In [ ]:
from sklearn.preprocessing import MinMaxScaler

trans = MinMaxScaler()

data = trans.fit_transform(descriptor_list)

##Lưu lại đặc trưng cho lần train tiếp theo

In [ ]:
import hickle as hck

hck.dump(descriptor_list, '/content/drive/My Drive/Machine Lol/backup/data3.hkl', mode='w')

hck.dump(labels, '/content/drive/My Drive/Machine Lol/backup/labels3.hkl', mode='w')

#Training model

###Load Data Backup(Nếu có)

In [ ]:
import hickle as hck
descriptor_list = hck.load('/content/drive/My Drive/Machine Lol/backup/data3.hkl')
labels = hck.load('/content/drive/My Drive/Machine Lol/backup/labels3.hkl')

print(len(descriptor_list))
print(len(labels))

3584
3584


##Chia tỉ lệ Train|Test
Tỉ lệ giữa data train và data test là 0.8|0.2

In [ ]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(data, labels,test_size =0.2, random_state=42)

##Model LogisticRegression

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support

lr = LogisticRegression()

lr.fit(trainX, trainY)


print(lr.score(trainX, trainY))
print(lr.score(testX, testY))

predY = lr.predict(testX)

print(precision_recall_fscore_support(testY, predY, average='macro'))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


1.0
0.8926080892608089
(0.891820857863751, 0.8938019504406494, 0.8923055915887212, None)


##Model GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()


gnb.fit(trainX, trainY)

print(gnb.score(trainX, trainY))
print(gnb.score(testX, testY))

predY = gnb.predict(testX)

print(precision_recall_fscore_support(testY, predY, average='macro'))

0.7111963725148238
0.6778242677824268
(0.6757473405094858, 0.6743030227133979, 0.6747239237296956, None)


##Model RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators= 10, random_state= 0)

rfc.fit(trainX, trainY)

print(rfc.score(trainX, trainY))
print(rfc.score(testX, testY))

predY = rfc.predict(testX)

print(precision_recall_fscore_support(testY, predY, average='macro'))

0.9919776770143006
0.701534170153417
(0.701454431053676, 0.6954627991797504, 0.6962008775125528, None)


##Model SVM

In [ ]:
from sklearn import svm

lsvc = svm.SVC()
lsvc.fit(trainX, trainY)

print(lsvc.score(trainX, trainY))
print(lsvc.score(testX, testY))

predY = lsvc.predict(testX)

print(precision_recall_fscore_support(testY, predY, average='macro'))

0.997558423439135
0.8870292887029289
(0.8865642046523293, 0.8860377565236448, 0.8862851828283322, None)


###Lưu Model

In [ ]:
import pickle

pickle.dump(lr, open('/content/drive/My Drive/Machine Lol/backup/logic3.sav', 'wb'))
pickle.dump(gnb, open('/content/drive/My Drive/Machine Lol/backup/gauss3.sav', 'wb'))
pickle.dump(rfc, open('/content/drive/My Drive/Machine Lol/backup/randforest3.sav', 'wb'))
pickle.dump(lsvc, open('/content/drive/My Drive/Machine Lol/backup/svm3.sav', 'wb'))

#Ứng dụng

###Load Model

In [5]:
import pickle

lr = pickle.load(open('/content/drive/My Drive/Machine Lol/backup/logic3.sav', 'rb'))
gnb = pickle.load(open('/content/drive/My Drive/Machine Lol/backup/gauss3.sav', 'rb'))
rfc = pickle.load(open('/content/drive/My Drive/Machine Lol/backup/randforest3.sav', 'rb'))
lsvc = pickle.load(open('/content/drive/My Drive/Machine Lol/backup/svm3.sav', 'rb'))

In [6]:
#@title Lựa chọn đầu vào
#@markdown Lựa chọn upload hay điền link video

Choose = "Link" #@param ["Upload", "Link", "---"]
#@markdown ---

Key = True
list_fn = []
if Choose == "Upload":
  from google.colab import files

  uploaded = files.upload()

  for fn in uploaded.keys():
    print('Người dùng đã upload file "{name}" độ lớn {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
    list_fn.append(fn)
elif Choose == "Link":
  print("Nhập link")
  link = input()

  import requests  
    
  r = requests.get(link, stream = True)  
  
  with open("/content/input.mp4", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
      if block:  
        file.write(block) 
  print("File đã được lưu với tên input.mp4")
  list_fn.append("input.mp4")

import numpy as np
import cv2
from skimage.feature import hog

num_vid = 0
while list_fn:
  i = list_fn.pop(0)
  cap = cv2.VideoCapture(i)

  ret, frame1 = cap.read()

  if ret:
    frames = []
    frames.append(frame1.copy())
    frame1 = cv2.resize(frame1, (160, 120))
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    s = 1
    roi = []
    while(cap.isOpened()):
        ret, frame2 = cap.read()
        if not ret:
          break;
        frames.append(frame2)
        if s % 6 == 0:
          frame2 = cv2.resize(frame2, (160, 120))
          #Trích xuất dòng quang từ 2 frame gần nhau
          next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
          optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
          flow = optical_flow.calc(prvs, next, None)
          mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
          hsv[...,0] = ang*180/np.pi/2
          nom = np.zeros((160,120))
          hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
          bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
          gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
          
          v = np.median(gray)
          sigma = 0.33
          lower = int(max(0, (1.0 - sigma) * v))
          upper = int(min(255, (1.0 + sigma) * v))
          canny = cv2.Canny(gray, lower, upper)
          

          kernel = np.ones((5,5),np.uint8)
          closing = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)

          contour,hier = cv2.findContours(closing,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

          for cnt in contour:
            cv2.drawContours(closing,[cnt],0,255,-1)

          opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)

          after_roi = next * opening

          # trích xuất feature bằng HOG
          image_hog = hog(after_roi, orientations=12, pixels_per_cell=(4, 4),
            cells_per_block=(2, 2), block_norm='L1', visualize=False)
          
          roi.append(image_hog)
          
          prvs = next.copy()


        s += 1
    Y = lsvc.predict(roi)
    h, w, d = frames[0].shape
    i = 0
    n = len(Y)
    while i < n:
      if Y[i] == 0:
        s = 5
        j = i*6
        while s > 0:
          cv2.rectangle(frames[j],(0,0),(w,h),(0,0,255),10)
          s -= 1
          j += 1
    out = cv2.VideoWriter('output'+str(num_vid)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, (w, h), True)
    for i in frames:
      out.write(i)
    
    out.release()
  num_vid += 1

Nhập link
https://github.com/UITTrinhQuangTruong/CS114.K21/raw/master/test3.mp4
File đã được lưu với tên input.mp4


KeyboardInterrupt: ignored

Tài liệu tham khảo
